In [435]:
import pandas as pd
import matplotlib.pyplot as plt

In [436]:
ex_rates = pd.read_csv("ex_rates.csv")
payments = pd.read_csv("payments.csv")
providers = pd.read_csv("providers.csv")

In [437]:
ex_rates.head(5)

,rate,destination
0,0.586865,AZN
1,1.041829,EUR
2,0.128461,HKD
3,0.000712,KRW
4,0.650000,AUD


In [438]:
payments['eventTimeRes'] = pd.to_datetime(payments['eventTimeRes'])
payments.head(5)

,eventTimeRes,amount,cur,payment,cardToken
0,2024-11-24 21:00:00,700.0,RUB,c77be33e8db2f5352ebbc0a1b3f10e3e,336d5ebc5436534e61d16e63ddfca327
1,2024-11-24 21:00:01,180.0,BRL,0ab79b6d41d6e887e84ec4a85ea33733,336d5ebc5436534e61d16e63ddfca327
2,2024-11-24 21:00:01,945.9,RUB,c9a03cbbe592f930d52416b049bc1d1a,336d5ebc5436534e61d16e63ddfca327
3,2024-11-24 21:00:01,35.0,BRL,90a88e0d5b6bfff12909878f265cc2f4,336d5ebc5436534e61d16e63ddfca327
4,2024-11-24 21:00:01,90.0,BRL,f3adde51de36a39508d11d4bf4a98edc,336d5ebc5436534e61d16e63ddfca327


In [439]:
providers.drop(columns=['LIMIT_BY_CARD'], inplace=True)
providers['TIME'] = pd.to_datetime(providers['TIME'])
providers.head(5)

,TIME,ID,CONVERSION,AVG_TIME,MIN_SUM,MAX_SUM,LIMIT_MIN,LIMIT_MAX,COMMISSION,CURRENCY
0,2024-11-24 21:00:00,0,0.50,18.0,700.0,5500.0,61000.0,12900000.0,0.035,UZS
1,2024-11-24 21:00:00,26,0.65,26.0,700.0,6000.0,31000.0,18600000.0,0.045,MYR
2,2024-11-24 21:00:00,28,0.65,22.0,200.0,9000.0,61000.0,20900000.0,0.045,MYR
3,2024-11-24 21:00:00,29,0.65,14.0,100.0,9000.0,81000.0,14700000.0,0.030,USD
4,2024-11-24 21:00:00,30,0.50,8.0,400.0,7000.0,71000.0,31600000.0,0.050,USD


TIME - дата и время, с которого состояние провайдера становится актуальным для платежей

ID - идентификатор провайдера

CONVERSION - идентификатор провайдера (отношение успешных к неуспешным платежам)

AVG_TIME - среднее время выполнения платежа на провайдере

MIN_SUM - минимальная сумма платежа (минимальный чек покупки)

MAX_SUM - максимальная сумма платежа (максимальный чек покупки)

LIMIT_MAX - максимальная общая сумма на провайдере за сутки (при превышении значения перестают проходить платежи)

LIMIT_MIN - минимальная общая сумма на провайдере за сутки (при недостижении значения получаем штрафы за невыработку объема)

LIMIT_BY_CARD - лимит по сумме платежей в разрезе одной карты плательщика

COMMISSION - комиссия, взимаемая провайдером

CURRENCY - валюта провайдера

Понять, какая разумная CONVERSION и AVG_TIME для провайдера. Это один из самых важных критериев: как же мы распределеим те или иные параметры (они зависят друг от друга). Опираться необходимо на цель заработать денег.

Лимит - важный параметр. Штраф - 1% от минимального лимита, если лимит не достигнут



In [440]:
providers_normalized = providers.copy()
numeric_columns = ['CONVERSION', 'AVG_TIME', 'COMMISSION']

for column in numeric_columns:
    min_val = providers_normalized[column].min()
    max_val = providers_normalized[column].max()
    providers_normalized[column] = (providers_normalized[column] - min_val) / (max_val - min_val)

In [441]:
providers['DATE'] = providers['TIME'].dt.date
limits = providers.groupby(['ID', 'DATE'])[['LIMIT_MIN', 'LIMIT_MAX']].first()
providers[['LIMIT_MIN', 'LIMIT_MAX']] = providers.apply(
    lambda row: limits.loc[(row['ID'], row['DATE'])], axis=1)
providers.drop(columns=['DATE'], inplace=True)

In [442]:
limits['LIMIT_CURR'] = 0
limits

LIMIT_MIN   LIMIT_MAX  LIMIT_CURR
ID DATE                                         
0  2024-11-24    61000.0  12900000.0           0
   2024-11-25    91000.0  15600000.0           0
   2024-11-26    11000.0  37400000.0           0
1  2024-11-24    21000.0  18000000.0           0
   2024-11-25    21000.0  15200000.0           0
...                  ...         ...         ...
45 2024-11-25    81000.0  33300000.0           0
46 2024-11-24    31000.0   8300000.0           0
   2024-11-25    61000.0  36200000.0           0
47 2024-11-24     1000.0  10100000.0           0
   2024-11-25    31000.0  12500000.0           0

[137 rows x 3 columns]

In [443]:
providers.sample(5)

,TIME,ID,CONVERSION,AVG_TIME,MIN_SUM,MAX_SUM,LIMIT_MIN,LIMIT_MAX,COMMISSION,CURRENCY
426,2024-11-25 17:00:00,29,0.75,26.0,100.0,4000.0,41000.0,17400000.0,0.035,USD
913,2024-11-26 20:00:00,32,0.65,18.0,700.0,3000.0,81000.0,30000000.0,0.030,HKD
823,2024-11-26 15:00:00,38,0.70,22.0,600.0,3000.0,61000.0,34800000.0,0.027,PHP
213,2024-11-25 06:00:00,12,0.60,26.0,300.0,5000.0,31000.0,18200000.0,0.040,TRY
863,2024-11-26 17:00:00,30,0.75,20.0,800.0,6500.0,81000.0,38400000.0,0.040,HKD


In [444]:
providers_normalized = providers.copy()
numeric_columns = ['CONVERSION', 'AVG_TIME', 'COMMISSION']

for column in numeric_columns:
    min_val = providers_normalized[column].min()
    max_val = providers_normalized[column].max()
    providers_normalized[column] = (providers_normalized[column] - min_val) / (max_val - min_val)

In [445]:
def provider_info(id_provider, payment, normalized=False):
    """
    Получает актуальную информацию о провайдере на момент платежа
    
    Args:
        id_provider (int): ID провайдера
        payment (pd.Series): Данные о платеже
        normalized (bool): Флаг, указывающий, нужно ли нормализовать данные
    Returns:
        pd.Series: Информация о провайдере
    """
    time = payment['eventTimeRes']
    if normalized:
        providers_data = providers_normalized.loc[providers_normalized['ID'] == id_provider].sort_values(by='TIME')
    else:
        providers_data = providers.loc[providers['ID'] == id_provider].sort_values(by='TIME')
    providers_data = providers_data.loc[providers_data['TIME'] <= time]
    provider_info = providers_data.iloc[-1]
    return provider_info
    

In [446]:
def metrics(payment, list_providers):
    """
    Рассчитывает метрики для платежа:
    conversion_metric - полная вероятность успешного платежа
    avg_time_metric - среднее время выполнения платежа при условии успешного платежа
    commission_metric - комиссия при условии успешного платежа

    Args:
        payment (pd.Series): Данные о платеже
        list_providers (list): Список ID провайдеров
    Returns:
        tuple: Метрики для платежа
    """
    conversions = [provider_info(list_providers[i], payment)['CONVERSION']
                    for i in range(len(list_providers))]
    avg_times = [provider_info(list_providers[i], payment)['AVG_TIME']
                    for i in range(len(list_providers))]
    commissions = [provider_info(list_providers[i], payment)['COMMISSION']
                    for i in range(len(list_providers))]
    
    conversion_metric = conversions[0]
    avg_time_metric = avg_times[0] * conversions[0]
    commission_metric = commissions[0] * conversions[0]

    probability = 1
    for i in range(1, len(list_providers)):
        probability *= conversions[i-1]
        conversion_metric += probability * conversions[i]
        avg_time_metric += probability * conversions[i] * avg_times[i]
        commission_metric += probability * conversions[i] * commissions[i]
        
    return conversion_metric, avg_time_metric, commission_metric

In [447]:
def grid_search_parameters():
    """
    Не сделал
    """
    best_params = {'q1': 0, 'q2': 0, 'q3': 0}
    best_score = 0
    for q1 in range(1, 1.1, 0.1):
        for q2 in range(1, 1.1, 0.1):
            for q3 in range(1, 1.1, 0.1):
                if q1 + q2 + q3 == 1:
                    pass

    return best_params

In [448]:
providers_indices = sorted(providers['ID'].unique())